In [2]:
import torch as t
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

In [3]:
# Device configuration
device = ('cuda' if t.cuda.is_available() else 'cpu')

In [4]:
batch_size = 100
num_epochs = 5
learning_rate = 0.01
num_classes = 10
input_size = 784

In [5]:
train_data = torchvision.datasets.MNIST(root="../../data/",
                                       train=True,
                                       download=True,
                                       transform=transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root="../../data/",
                                       train=False,
                                       transform=transforms.ToTensor())

In [21]:
train_loader = t.utils.data.DataLoader(dataset=train_data,
                                      shuffle=True,
                                      batch_size=batch_size)
test_loader = t.utils.data.DataLoader(dataset=test_data,
                                      shuffle=False,
                                      batch_size=batch_size)

In [18]:
# Convolutional neural network (two convolutional layers)
class CNN(nn.Module):
    def __init__(self, num_classes):
        
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2), #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
            nn.BatchNorm2d(16), #torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), #torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(7*7*32, num_classes)
        
    def forward(self, X):
        out = self.layer1(X)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return(out)

In [19]:
# Initilize model

model = CNN(num_classes) # this is the same as CNN.forward()
criterion = nn.CrossEntropyLoss()
optimizer = t.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
# Train model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        pred = model(imgs)
        
        loss = criterion(pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0898
Epoch [1/5], Step [200/600], Loss: 0.1163
Epoch [1/5], Step [300/600], Loss: 0.0429
Epoch [1/5], Step [400/600], Loss: 0.0239
Epoch [1/5], Step [500/600], Loss: 0.0131
Epoch [1/5], Step [600/600], Loss: 0.0214
Epoch [2/5], Step [100/600], Loss: 0.0236
Epoch [2/5], Step [200/600], Loss: 0.0120
Epoch [2/5], Step [300/600], Loss: 0.0351
Epoch [2/5], Step [400/600], Loss: 0.0182
Epoch [2/5], Step [500/600], Loss: 0.0163
Epoch [2/5], Step [600/600], Loss: 0.0017
Epoch [3/5], Step [100/600], Loss: 0.0454
Epoch [3/5], Step [200/600], Loss: 0.0187
Epoch [3/5], Step [300/600], Loss: 0.0081
Epoch [3/5], Step [400/600], Loss: 0.0221
Epoch [3/5], Step [500/600], Loss: 0.0056
Epoch [3/5], Step [600/600], Loss: 0.0078
Epoch [4/5], Step [100/600], Loss: 0.0033
Epoch [4/5], Step [200/600], Loss: 0.0156
Epoch [4/5], Step [300/600], Loss: 0.0003
Epoch [4/5], Step [400/600], Loss: 0.0023
Epoch [4/5], Step [500/600], Loss: 0.0203
Epoch [4/5], Step [600/600], Loss:

In [24]:
# Test Model
model.eval() # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with t.no_grad():
    total = correct = 0
    for i , (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
    
        outputs = model(images)
        _, predicted = t.max(outputs.data, 1)
        
        total += labels.size(0)
        correct+= (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
    
t.save(model.state_dict(), "CNN.ckpt")

Test Accuracy of the model on the 10000 test images: 99.18 %
